# Day12_3: 첫 ML 모델 (First ML Model)

## 📚 학습 목표

**Part 1: 기초 - Titanic 데이터 이해**
1. Titanic 데이터셋의 구조와 의미 파악하기
2. EDA(탐색적 데이터 분석)로 생존율 패턴 발견하기
3. 결측치 처리 및 범주형 변수 인코딩하기
4. 특성(Feature)과 타겟(Target) 분리하기
5. 훈련/테스트 데이터 분할하기

**Part 2: 심화 - 로지스틱 회귀 모델**
1. LogisticRegression 모델 훈련하기
2. 예측 및 확률 계산하기
3. 정확도, 정밀도, 재현율, F1 스코어 평가하기
4. Plotly로 모델 성능 시각화하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 이진 분류 | 예/아니오 예측 문제 | 고객 이탈, 사기 탐지, 스팸 필터링 |
| 로지스틱 회귀 | 해석 가능한 기본 모델 | 의료 진단, 신용 평가 |
| 특성 엔지니어링 | 모델 성능 향상의 핵심 | 범주형 인코딩, 결측치 처리 |
| 성능 평가 | 모델 품질 검증 | 정확도, F1 스코어, 혼동 행렬 |

**분석가 관점**: Titanic 생존 예측은 ML 입문의 "Hello World"입니다. 이 프로젝트로 전체 ML 파이프라인을 경험합니다!

---

## 환경 설정

In [ ]:
# 필수 라이브러리
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

# Part 1: 기초 - Titanic 데이터 이해

---

## 1.1 데이터 로드 및 구조 파악

In [ ]:
# Titanic 데이터 로드 (seaborn 내장 데이터셋 활용)
import seaborn as sns
titanic = sns.load_dataset('titanic')

print(f"데이터 크기: {titanic.shape[0]}행 x {titanic.shape[1]}열")
titanic.head()

In [ ]:
# 컬럼 설명
column_desc = {
    'survived': '생존 여부 (0=사망, 1=생존) - 타겟 변수',
    'pclass': '객실 등급 (1=1등석, 2=2등석, 3=3등석)',
    'sex': '성별',
    'age': '나이',
    'sibsp': '함께 탑승한 형제자매/배우자 수',
    'parch': '함께 탑승한 부모/자녀 수',
    'fare': '운임',
    'embarked': '탑승 항구 (C=Cherbourg, Q=Queenstown, S=Southampton)',
    'class': '객실 등급 (문자형)',
    'who': '남자/여자/아이',
    'adult_male': '성인 남성 여부',
    'deck': '갑판',
    'embark_town': '탑승 도시',
    'alive': '생존 여부 (문자형)',
    'alone': '혼자 탑승 여부'
}

for col, desc in column_desc.items():
    print(f"{col:15} : {desc}")

In [ ]:
# 기본 통계
titanic.describe()

In [ ]:
# 결측치 확인
missing = titanic.isnull().sum()
missing_pct = (missing / len(titanic) * 100).round(1)

missing_df = pd.DataFrame({
    '결측치 수': missing,
    '비율(%)': missing_pct
}).query('`결측치 수` > 0').sort_values('결측치 수', ascending=False)

print("결측치 현황:")
missing_df

### 💡 실무 예시: 결측치 처리 전략

- **age (19.9%)**: 중앙값 대체 (이상치에 강건)
- **deck (77.2%)**: 결측 비율이 높아 삭제
- **embarked (0.2%)**: 최빈값 대체

---

## 1.2 EDA (탐색적 데이터 분석)

In [ ]:
# 전체 생존율
survival_rate = titanic['survived'].mean() * 100
print(f"전체 생존율: {survival_rate:.1f}%")

In [ ]:
# 성별에 따른 생존율
sex_survival = titanic.groupby('sex')['survived'].agg(['mean', 'count'])
sex_survival.columns = ['생존율', '인원수']
sex_survival['생존율'] = (sex_survival['생존율'] * 100).round(1)
print("성별 생존율:")
sex_survival

In [ ]:
# 성별 생존율 시각화 (Plotly)
fig = px.bar(
    titanic.groupby('sex')['survived'].mean().reset_index(),
    x='sex', y='survived',
    color='sex',
    title='성별에 따른 생존율',
    labels={'sex': '성별', 'survived': '생존율'},
    text_auto='.1%'
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# 객실 등급에 따른 생존율
pclass_survival = titanic.groupby('pclass')['survived'].mean().reset_index()

fig = px.bar(
    pclass_survival,
    x='pclass', y='survived',
    color='pclass',
    title='객실 등급에 따른 생존율',
    labels={'pclass': '객실 등급', 'survived': '생존율'},
    text_auto='.1%'
)
fig.show()

In [ ]:
# 나이 분포 및 생존 여부
fig = px.histogram(
    titanic,
    x='age',
    color='survived',
    nbins=30,
    title='나이별 생존 분포',
    labels={'age': '나이', 'survived': '생존'},
    barmode='overlay',
    opacity=0.7
)
fig.show()

### 💡 EDA 인사이트

1. **여성**의 생존율이 남성보다 훨씬 높음 (약 74% vs 19%)
2. **1등석** 승객의 생존율이 가장 높음 (약 63%)
3. **어린이**의 생존율이 상대적으로 높음

---

## 1.3 특성 엔지니어링

In [ ]:
# 분석용 데이터 복사
df = titanic.copy()

# 사용할 특성 선택 (간단한 모델을 위해)
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
target = 'survived'

df = df[features + [target]]
print(f"선택된 특성: {features}")
df.head()

In [ ]:
# 1. 결측치 처리
# age: 중앙값으로 대체
df['age'] = df['age'].fillna(df['age'].median())

# embarked: 최빈값으로 대체
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

print("결측치 처리 후:")
print(df.isnull().sum())

In [ ]:
# 2. 범주형 변수 인코딩
# sex: 이진 인코딩 (male=0, female=1)
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# embarked: 원-핫 인코딩
df = pd.get_dummies(df, columns=['embarked'], prefix='embarked')

print("인코딩 후 컬럼:")
print(df.columns.tolist())
df.head()

### 💡 실무 예시: 인코딩 방법 선택

| 변수 유형 | 인코딩 방법 | 예시 |
|----------|------------|------|
| 이진 범주형 | Label Encoding | 성별 (남/여) |
| 다중 범주형 (순서 없음) | One-Hot Encoding | 탑승 항구 |
| 다중 범주형 (순서 있음) | Ordinal Encoding | 학력 (초/중/고/대) |

---

## 1.4 데이터 분할

In [ ]:
# 특성(X)과 타겟(y) 분리
X = df.drop(columns=[target])
y = df[target]

print(f"특성 (X): {X.shape}")
print(f"타겟 (y): {y.shape}")

In [ ]:
# 훈련/테스트 데이터 분할 (80:20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20%를 테스트로
    random_state=42,    # 재현성
    stratify=y          # 클래스 비율 유지
)

print(f"훈련 세트: {X_train.shape[0]}개")
print(f"테스트 세트: {X_test.shape[0]}개")
print(f"\n훈련 세트 생존율: {y_train.mean():.1%}")
print(f"테스트 세트 생존율: {y_test.mean():.1%}")

### 💡 실무 예시: stratify 파라미터

`stratify=y`를 사용하면 훈련/테스트 세트에서 클래스 비율이 동일하게 유지됩니다.
불균형 데이터에서 특히 중요합니다.

---

# Part 2: 심화 - 로지스틱 회귀 모델

---

## 2.1 모델 훈련

In [ ]:
# 로지스틱 회귀 모델 생성 및 훈련
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

print("모델 훈련 완료!")

In [ ]:
# 특성 중요도 확인 (계수)
feature_importance = pd.DataFrame({
    '특성': X.columns,
    '계수': model.coef_[0]
}).sort_values('계수', key=abs, ascending=False)

print("특성별 계수 (절댓값 기준 정렬):")
feature_importance

In [ ]:
# 특성 중요도 시각화
fig = px.bar(
    feature_importance,
    x='계수', y='특성',
    orientation='h',
    title='로지스틱 회귀 특성 계수',
    color='계수',
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

### 💡 계수 해석

- **양수 계수**: 값이 증가하면 생존 확률 증가 (예: sex=1(여성))
- **음수 계수**: 값이 증가하면 생존 확률 감소 (예: pclass 증가 = 등급 낮아짐)

---

## 2.2 예측

In [ ]:
# 클래스 예측 (0 또는 1)
y_pred = model.predict(X_test)

print("예측 결과 (처음 10개):")
print(f"실제: {y_test.values[:10]}")
print(f"예측: {y_pred[:10]}")

In [ ]:
# 확률 예측
y_prob = model.predict_proba(X_test)

print("확률 예측 (처음 5개):")
print("[사망 확률, 생존 확률]")
print(y_prob[:5].round(3))

In [ ]:
# 예측 결과 DataFrame
result_df = X_test.copy()
result_df['실제'] = y_test.values
result_df['예측'] = y_pred
result_df['생존확률'] = y_prob[:, 1].round(3)

result_df.head(10)

---

## 2.3 성능 평가

In [ ]:
# 기본 평가 지표
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("=" * 40)
print("모델 성능 평가")
print("=" * 40)
print(f"정확도 (Accuracy):  {accuracy:.3f}")
print(f"정밀도 (Precision): {precision:.3f}")
print(f"재현율 (Recall):    {recall:.3f}")
print(f"F1 스코어:          {f1:.3f}")

In [ ]:
# 혼동 행렬 (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)

print("혼동 행렬:")
print(f"          예측 0   예측 1")
print(f"실제 0    {cm[0,0]:5d}    {cm[0,1]:5d}")
print(f"실제 1    {cm[1,0]:5d}    {cm[1,1]:5d}")

In [ ]:
# 혼동 행렬 시각화 (Plotly)
fig = px.imshow(
    cm,
    labels=dict(x="예측", y="실제", color="건수"),
    x=['사망 (0)', '생존 (1)'],
    y=['사망 (0)', '생존 (1)'],
    text_auto=True,
    color_continuous_scale='Blues',
    title='혼동 행렬 (Confusion Matrix)'
)
fig.show()

### 💡 평가 지표 해석

| 지표 | 의미 | 사용 상황 |
|------|------|----------|
| 정확도 | 전체 중 맞춘 비율 | 클래스 균형일 때 |
| 정밀도 | 생존 예측 중 실제 생존 비율 | 오탐(FP)이 중요할 때 |
| 재현율 | 실제 생존 중 예측 성공 비율 | 미탐(FN)이 중요할 때 |
| F1 | 정밀도와 재현율의 조화평균 | 균형 잡힌 평가 |

---

## 2.4 결과 시각화

In [ ]:
# 평가 지표 비교 차트
metrics = pd.DataFrame({
    '지표': ['정확도', '정밀도', '재현율', 'F1 스코어'],
    '값': [accuracy, precision, recall, f1]
})

fig = px.bar(
    metrics,
    x='지표', y='값',
    title='모델 성능 지표 비교',
    text_auto='.3f',
    color='지표'
)
fig.update_layout(showlegend=False, yaxis_range=[0, 1])
fig.show()

In [ ]:
# 예측 확률 분포
prob_df = pd.DataFrame({
    '생존확률': y_prob[:, 1],
    '실제': y_test.map({0: '사망', 1: '생존'}).values
})

fig = px.histogram(
    prob_df,
    x='생존확률',
    color='실제',
    nbins=30,
    title='예측 확률 분포 (실제 클래스별)',
    barmode='overlay',
    opacity=0.7
)
fig.add_vline(x=0.5, line_dash="dash", line_color="red", annotation_text="임계값 0.5")
fig.show()

In [ ]:
# 특성별 생존 확률 분석
# 성별 x 객실등급별 생존율
survival_matrix = titanic.groupby(['sex', 'pclass'])['survived'].mean().unstack()

fig = px.imshow(
    survival_matrix,
    labels=dict(x="객실 등급", y="성별", color="생존율"),
    text_auto='.1%',
    color_continuous_scale='RdYlGn',
    title='성별 x 객실등급별 생존율 히트맵'
)
fig.show()

### 💡 시각화 인사이트

1. 여성 1등석 승객의 생존율이 가장 높음 (약 97%)
2. 남성 3등석 승객의 생존율이 가장 낮음 (약 14%)
3. 성별이 객실 등급보다 더 강력한 생존 예측 인자

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 데이터 탐색 ⭐

**문제**: titanic 데이터에서 생존자(survived=1) 수와 사망자(survived=0) 수를 각각 출력하세요.

**힌트**: `value_counts()` 메소드 사용

In [ ]:
# 데이터 다시 로드
import seaborn as sns
titanic = sns.load_dataset('titanic')

# 여기에 코드를 작성하세요


### Q2. 그룹별 생존율 ⭐

**문제**: 객실 등급(pclass)별 평균 생존율을 계산하세요.

**힌트**: `groupby()` + `mean()`

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 결측치 처리 ⭐⭐

**문제**: age 컬럼의 결측치를 평균값으로 대체하고, 결측치가 모두 처리되었는지 확인하세요.

**힌트**: `fillna()` + `mean()`

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 범주형 인코딩 ⭐⭐

**문제**: sex 컬럼을 숫자로 인코딩하세요 (male=0, female=1).

**힌트**: `map()` 메소드

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 데이터 분할 ⭐⭐

**문제**: X와 y로 분리한 후, 70:30 비율로 훈련/테스트 세트를 나누세요. (random_state=42)

**힌트**: `train_test_split()` 함수

In [ ]:
# 전처리된 데이터 준비
df = titanic[['pclass', 'sex', 'age', 'fare', 'survived']].copy()
df['age'] = df['age'].fillna(df['age'].median())
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

X = df.drop(columns=['survived'])
y = df['survived']

# 여기에 코드를 작성하세요 (70:30 분할)


### Q6. 모델 훈련 ⭐⭐⭐

**문제**: LogisticRegression 모델을 생성하고 훈련한 후, 훈련 세트에서의 정확도를 출력하세요.

**힌트**: `model.score(X_train, y_train)`

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 예측 수행 ⭐⭐⭐

**문제**: 테스트 세트에 대해 클래스 예측과 확률 예측을 수행하세요.

**힌트**: `predict()` + `predict_proba()`

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 성능 평가 ⭐⭐⭐⭐

**문제**: 테스트 세트에서 정확도, 정밀도, 재현율, F1 스코어를 모두 계산하세요.

**힌트**: `accuracy_score`, `precision_score`, `recall_score`, `f1_score`

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 혼동 행렬 시각화 ⭐⭐⭐⭐

**문제**: 테스트 세트의 혼동 행렬을 계산하고 Plotly로 시각화하세요.

**힌트**: `confusion_matrix()` + `px.imshow()`

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 새로운 승객 정보가 주어졌을 때, 생존 확률을 예측하는 함수를 작성하세요.

```python
def predict_survival(pclass, sex, age, fare):
    # 승객 정보를 받아 생존 확률 반환
    pass

# 테스트: 1등석, 여성, 25세, 운임 100
prob = predict_survival(pclass=1, sex='female', age=25, fare=100)
print(f"생존 확률: {prob:.1%}")
```

**힌트**: 입력값을 모델에 맞게 변환 후 `predict_proba()` 사용

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 단계 | 핵심 내용 | 주요 함수/메소드 |
|------|----------|------------------|
| 데이터 로드 | 데이터셋 불러오기 | `pd.read_csv()`, `sns.load_dataset()` |
| EDA | 패턴 발견, 시각화 | `groupby()`, `value_counts()`, `px.bar()` |
| 결측치 처리 | 중앙값/최빈값 대체 | `fillna()`, `median()`, `mode()` |
| 인코딩 | 범주형 → 숫자 | `map()`, `get_dummies()` |
| 데이터 분할 | 훈련/테스트 분리 | `train_test_split()` |

### Part 2: 심화 핵심 요약

| 단계 | 핵심 내용 | 주요 함수/메소드 |
|------|----------|------------------|
| 모델 훈련 | 로지스틱 회귀 학습 | `LogisticRegression().fit()` |
| 예측 | 클래스 및 확률 예측 | `predict()`, `predict_proba()` |
| 평가 | 4대 지표 계산 | `accuracy_score`, `precision_score`, `recall_score`, `f1_score` |
| 시각화 | 결과 분석 | `confusion_matrix()`, `px.imshow()`, `px.bar()` |

### 💡 실무 팁

1. **EDA 먼저**: 모델링 전 반드시 데이터 탐색 (결측치, 분포, 패턴)
2. **stratify 사용**: 불균형 데이터에서 클래스 비율 유지
3. **여러 지표 확인**: 정확도만 보지 말고 정밀도/재현율도 확인
4. **혼동 행렬 분석**: FP/FN의 비즈니스 영향 고려
5. **기준선(Baseline) 설정**: 단순 모델부터 시작해 점진적 개선